In [73]:
# make sure we have pandas
%%capture
!pip3 install pandas

In [74]:
# import pandas with alias and other libraries allowing for loading of json data
import pandas as pd
import urllib.request,json

In [76]:
#declare paths to external files online
building_footprints_path="https://services1.arcgis.com/KNdRU5cN6ENqCTjk/arcgis/rest/services/CSU_Building_Footprints/FeatureServer/0//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
haunted_csu_path="https://docs.google.com/spreadsheets/d/e/2PACX-1vTilTLQU-L6FRT75GIw_TT0SMWa7b3QHqWiY4HCBx_MGZcPyYb_NXKpdv7DBhCEtkAW-GZW9XP6zazv/pub?gid=1226537332&single=true&output=csv"
# for reference - here is a file will all the building histories
# building_histories_path="https://docs.google.com/spreadsheets/d/e/2PACX-1vTilTLQU-L6FRT75GIw_TT0SMWa7b3QHqWiY4HCBx_MGZcPyYb_NXKpdv7DBhCEtkAW-GZW9XP6zazv/pub?gid=1626181908&single=true&output=csv,csv"

In [77]:
# load in the haunted building spreadsheet
haunted_csu=pd.read_csv(haunted_csu_path)
haunted_csu

,Name,Description,Images,Reference
0,Old Main,"Mysteriously burned down overnight on May 8, 1...",https://libapps.s3.amazonaws.com/accounts/1539...,https://source.colostate.edu/hallowed-haunted-...
1,Johnson Hall,Once the location of the first Student Union c...,https://libapps.s3.amazonaws.com/accounts/1539...,https://source.colostate.edu/hallowed-haunted-...
2,Student Services Building,"Built in 1948, and formally known as the Braid...",https://libapps.s3.amazonaws.com/accounts/1539...,https://source.colostate.edu/hallowed-haunted-...
3,Ammons Hall,Originally the female Recreation Center at the...,https://archives.mountainscholar.org/digital/a...,https://source.colostate.edu/hallowed-haunted-...


In [78]:
# load in a geojson file with CSU building footprints
with urllib.request.urlopen(building_footprints_path) as url:
    building_footprints = json.load(url)
building_footprints

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 1,
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-105.080470479183, 40.5779231618761],
      [-105.080346280415, 40.5779227119488],
      [-105.080346434084, 40.5778980083481],
      [-105.080313806285, 40.5778979654639],
      [-105.080314119795, 40.5778841667309],
      [-105.080206120929, 40.5778837753511],
      [-105.080206479609, 40.5778261336152],
      [-105.080136268818, 40.5778245832228],
      [-105.080138435825, 40.5777655750474],
      [-105.080206855367, 40.5777657470343],
      [-105.080207419004, 40.5776751671614],
      [-105.080221818808, 40.5776752193513],
      [-105.080221912744, 40.5776601227054],
      [-105.080236289634, 40.5776602488735],
      [-105.08058187721, 40.5776614972081],
      [-105.080598107545, 40.5776614855279],
      [-105.080598022226, 40.577675209751],
      [-105.08061242203, 40.5776752618922],
      [-105.080611116681, 40.5778852425051],
      [-105.0804995178

In [79]:
# create a subset of the building footprints that are haunted
subset={'type': 'FeatureCollection','features': []}
for feature in building_footprints["features"]:
    # print(features["properties"]['NAME_LC'])
    for index, haunted in haunted_csu.iterrows():
        if (haunted["Name"]==feature["properties"]['NAME_LC']):
            subset["features"].append(feature)

In [82]:
# we'll now start creating a map
# ref: https://carpentries-incubator.github.io/jupyter_maps/01-introduction/index.html
# start by installing the web-map library
%%capture
!pip3 install ipyleaflet

UsageError: Line magic function `%%capture` not found.


In [90]:
# import the web-map library and a jupyter widges for displaying a popup with HTMl
from ipyleaflet import Map, GeoJSON
from ipywidgets import HTML

In [84]:
# to diaplay an embeded web-map a couple of extensions are required
#note - you will need to refresh the notebook to load in the javascript
%%capture
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install jupyter-leaflet

UsageError: Line magic function `%%capture` not found.


In [91]:
# create a map object, centered over out regio
map = Map(center = (40.5762663,-105.0816224,), zoom = 16, min_zoom = 1, max_zoom = 20)

# create a geojson layer
geo_json = GeoJSON(
    data=subset,
    style={
        'color': 'green','opacity': 1, 'dashArray': '9', 'fillOpacity': 0.2, 'weight': 2
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },popup=HTML("boo")
)
#add the layer
map.add_layer(geo_json)
# show the map
display(map)


Map(center=[40.5762663, -105.0816224], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…